In [1]:
import sys
sys.path.insert(0,"F:\\repos\\phd\\src")

In [2]:
import os
import pydicom

import matplotlib.pyplot as plt
import numpy as np

In [3]:
from outcomeDL.preprocessing import extract_arrays as ext
from outcomeDL.preprocessing import _preprocess_util as util

In [4]:
def load_files(folder):
    files = [os.path.join(folder,file) for file in os.listdir(folder) if file.endswith(".dcm")]
    sorted_files = {}
    for filepath in files:
        dcm = pydicom.read_file(filepath)
        m = dcm.Modality
        if m not in sorted_files.keys():
            sorted_files[m] = [dcm]
        else:
            sorted_files[m].append(dcm)
    return sorted_files

def all_one_study(filedict):
    UID = []
    for key in filedict.keys():
        for file in filedict[key]:
            UID.append(file.StudyInstanceUID)
    UID = list(set(UID))
    if len(UID) > 1:
        clean = False
    elif len(UID) == 1:
        clean = True
    return clean

def correct_array_shapes(filelist):
    """
    Function which trims arrays of non-uniform shape to largest common array (fits all arrays).
    Requires that all arrays in the list have the same corner position (ImagePositionPatient attribute).
    
    Parameters
    ----------
    dosefilelist : list of DICOM files
        List containing pydicom loaded DICOM objects.
    
    Returns
    -------
    dosefilelist : list of DICOM files
        Same list, but with the files modified to be shape-compatible.
    """
    if not same_position(filelist):
        raise Exception("Files not corner-aligned, can't match them")
    # first find the smallest array - note that pixel_array.shape is structured as Z, Rows, Columns
    min_row = np.inf
    min_col = np.inf
    min_z = np.inf
    # first loop through all to find the min for each axis
    for file in filelist:
        z, rows, cols = file.pixel_array.shape
        min_row = min(rows, min_row)
        min_col = min(cols, min_col)
        min_z = min(z, min_z)
    # next restructure each array as needed
    for file in filelist:
        new_array = file.pixel_array
        if file.pixel_array.shape[0] > min_z:
            new_array = new_array[:min_z,:,:]
            file.GridFrameOffsetVector = file.GridFrameOffsetVector[:min_z]
        if file.pixel_array.shape[1] > min_row:
            new_array = new_array[:,:min_row,:]
            file.Rows = min_row
        if file.pixel_array.shape[2] > min_col:
            new_array = new_array[:,:,:min_col]
            file.Columns = min_col
        file.PixelData = new_array.tobytes()
    return filelist
        
def same_position(dicom_list):
    c,r,z = dicom_list[0].ImagePositionPatient
    for obj in dicom_list:
        tc, tr, tz = obj.ImagePositionPatient
        if not all((tc == c, tr == r, tz == z)):
            return False
    return True

In [5]:
badIDs = ['018_019','018_033','018_038','018_056','018_069','018_089',
          '018_091','018_102','018_117','018_120',
          '018_126','018_128','018_130','018_131',
          'ANON_016','ANON_023','ANON_027','018_011']

In [6]:
sourcedir = "F:\\DICOMdata\\RoswellData"
#sourcedir = "D:\\H_N"

In [12]:
patientpath = os.path.join(sourcedir,'018_102')
files = load_files(patientpath)

In [13]:
imgarray, slicemap, imcorner = ext.prepare_image_array(files['CT'])
dose, true_z, dosecorner = ext.prepare_dose_array(files['RTDOSE'])

In [ ]:
test = correct_array_shapes(files['RTDOSE'])

In [ ]:
shapes = [file.pixel_array.shape for file in test]
print(len(set(shapes)))

In [ ]:
len(files['RTDOSE'])

In [ ]:
util.all_one_study(files)

In [15]:
dosecorner

[-250.6, -121.0]